In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
%cd '/gdrive/MyDrive/SentimentAnalysis'

/gdrive/MyDrive/SentimentAnalysis


In [7]:
!ls

Untitled0.ipynb


In [8]:
import numpy as np
import pandas as pd

In [10]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 
     |████████████████████████████████| 3.3 MB 42.0 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 636 kB 41.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Instanceciate the Model (Bert)

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  


In [12]:
# Preprocessor (converts into word tokens)
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# Model
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

**Trying on sample sentence**

In [174]:
# Create batch with data
pt_batch = tokenizer(
    ["We like your product but it has faults"],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
 )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [175]:
# Give batch to model and get result
results = model(**pt_batch)

In [176]:
# argmax taking the index of the maxvalue tensor(which is our sentiment) and adding one bcz array was from 0 to 1
t = int(results.logits.argmax()+1)

In [177]:
results

SequenceClassifierOutput([('logits',
                           tensor([[-1.5985, -0.1479,  1.5766,  1.1208, -0.8350]],
                                  grad_fn=<AddmmBackward>))])

In [178]:
t

3

# WebScraping

**From TMDB**

In [57]:
!pip install beautifulsoup4

In [60]:
!pip install requests

In [117]:
from bs4 import BeautifulSoup
import requests
import re


In [161]:
# Making request
# Headers to bypass 403 error
movieid= 1911
# name = 'batman'
# cleanname = re.sub(" ","-",name)
headers = {
  'User-Agent':
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
}
URL = 'https://www.themoviedb.org/movie/{}/reviews'.format(movieid)

makerequest = requests.get(URL,headers = headers)
# Make Soup
soup  = BeautifulSoup(makerequest.text,  'html.parser')
# Make regex 
regex = re.compile('.*teaser*.')
# Find all componenets where where you find regex in class of div eg <div class=teaser>
result = soup.find_all('div',{'class' : regex})
# Remove unnecessary tags
resultsclean  = [i.text for i in result]

In [162]:
resultsclean[0]

'\nGood Viking Adventure -- Could\'ve Been Great\nThe story of "The 13th Warrior" (1999) comes from Michael Crichton\'s novel "The Eaters of the Dead" which combines the legend of Beowulf with a historical account of an Arab diplomat who meets and dwells with the Vikings after being banished from his homeland due to an adulterous liaison.\nAntonio Banderas stars as the Arab while the hulking Vladimir Kulich heads the Viking cast in the Beowulf role of Buliwyf (pronounced in the film BULL-vie). Speaking of Kulich, he would\'ve made for an excellent Mighty Thor back in the day!\nThe plot of... read the rest.\n'

In [163]:
# Remove "/n" and "read rest"
t = [re.sub('\n',"", i) for i in resultsclean]
t = [re.sub('read the rest',"", i) for i in t]


In [165]:
t

['Good Viking Adventure -- Could\'ve Been GreatThe story of "The 13th Warrior" (1999) comes from Michael Crichton\'s novel "The Eaters of the Dead" which combines the legend of Beowulf with a historical account of an Arab diplomat who meets and dwells with the Vikings after being banished from his homeland due to an adulterous liaison.Antonio Banderas stars as the Arab while the hulking Vladimir Kulich heads the Viking cast in the Beowulf role of Buliwyf (pronounced in the film BULL-vie). Speaking of Kulich, he would\'ve made for an excellent Mighty Thor back in the day!The plot of... .']

# Sentiments on Custom Webscraped Data

In [215]:
# Making request
# Headers to bypass 403 error

# name = 'batman'
# cleanname = re.sub(" ","-",name)
movieid= 272
headers = {
  'User-Agent':
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
}
URL = 'https://www.themoviedb.org/movie/{}/reviews'.format(movieid)

makerequest = requests.get(URL,headers = headers)
# Make Soup
soup  = BeautifulSoup(makerequest.text,  'html.parser')
# Make regex 
regex = re.compile('.*teaser*.')
# Find all componenets where where you find regex in class of div eg <div class=teaser>
result = soup.find_all('div',{'class' : regex})
# Remove unnecessary tags
resultsclean  = [i.text for i in result]
t = [re.sub('\n',"", i) for i in resultsclean]
t = [re.sub('read the rest',"", i) for i in t]                                                                                    

In [254]:
# Getting Sentiment Scores for all the comments
res = []
response = []
neutral = []
bad = []
good = []
output = pd.DataFrame()

for i in t:
    print('h',i)
    # Create batch with data
    pt_batch = tokenizer(i,truncation=True,max_length=512,return_tensors="pt")
    scores = model(**pt_batch)
    # res.append( int(scores.logits.argmax()+1))
    score = int(scores.logits.argmax()+1)
    if( score == 3  or score == 4 ):
      neutral.append({'comment' : i , 'sentiment': 'Just Okay'})
      
         
    if(score == 1 or  score == 2 ):
      bad.append({'comment' : i , 'sentiment': 'Just Okay'})
      

         
    if(score == 5 ):
          good.append({'comment' : i , 'sentiment': 'Fantastic'})
          

res =  {'neutral' : neutral , 'good' : good  , 'bad': bad}



   
 
    
  

  

h It's not who I am underneath, but what I do that defines me.Bruce Wayne is constantly tortured by his childhood memories when he witnessed his parents being murdered. Taken under the wing of The League Of Shadows, a deadly ninja assassin army devoted to erasing crime with their own brand of harsh justice. After completing training, Wayne refuses to join them on account of not agreeing with their methods, he returns to Gotham City to reek his own one man war against crime.Director Christopher Nolan literally goes back to Batman origins to not just give the dead franchise a kiss of life,... .
h i'm beginning to think people only say a movie is good because of it being long and having a good camera. to me the only highlight was cillian murphy's performance. the rest of this was just some dumb, boring trek.
h Christopher Nolan's "Batman Begins" shows us exactly how Bruce Wayne (Christian Bale) transforms into the caped crusader we all know and love, Batman. Admittedly like many others, e

In [255]:
res

{'response': {'bad': [{'comment': "i'm beginning to think people only say a movie is good because of it being long and having a good camera. to me the only highlight was cillian murphy's performance. the rest of this was just some dumb, boring trek.",
    'sentiment': 'Just Okay'}],
  'good': [],
  'neutral': [{'comment': "It's not who I am underneath, but what I do that defines me.Bruce Wayne is constantly tortured by his childhood memories when he witnessed his parents being murdered. Taken under the wing of The League Of Shadows, a deadly ninja assassin army devoted to erasing crime with their own brand of harsh justice. After completing training, Wayne refuses to join them on account of not agreeing with their methods, he returns to Gotham City to reek his own one man war against crime.Director Christopher Nolan literally goes back to Batman origins to not just give the dead franchise a kiss of life,... .",
    'sentiment': 'Just Okay'},
   {'comment': 'Christopher Nolan\'s "Batman

# JOblib

In [257]:
import joblib

In [258]:
joblib.dump(tokenizer,'tokenizer.joblib')

['tokenizer.joblib']

In [259]:
joblib.dump(model,'model.joblib')

['model.joblib']